<h1>Part 3 of 3: Extracting and Visualizing Stock Data</h1>
<h2>Description</h2>


Extracting essential data from a dataset and displaying it is a necessary part of data science; therefore individuals can make correct decisions based on the data. In this project, we will extract some stock data and then display the data in a graph.


<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Define a Function that Makes a Graph</li>
        <li>Use yfinance to Extract Stock Data</li>
        <li>Use Webscraping to Extract Tesla Revenue Data</li>
        <li>Use yfinance to Extract Stock Data</li>
        <li>Use Webscraping to Extract GME Revenue Data</li>
        <li>Plot Tesla Stock Graph</li>
        <li>Plot GameStop Stock Graph</li>
    </ul>
<p>
</div>

<hr>


In [ ]:
# !pip install yfinance==0.1.67
# !mamba install bs4==4.10.0 -y
# !pip install nbformat==4.2.0

In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Surpressing warnings:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn


## Defining Graphing Function


In this section, we define the function `make_graph`. You don't have to know how the function works, you should only care about the inputs. It takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock. [PRE MADE]


In [ ]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.


In [ ]:
ts= yf.Ticker('TSLA')

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to `max` so we get information for the maximum amount of time.


In [ ]:
tesla_data=ts.history(period= "max")

**Reset the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and display the first five rows of the `tesla_data` dataframe using the `head` function. 

In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

## Using Webscraping to Extract Tesla Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm 

In [ ]:
x= requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm")
html_data= x.text

Parse the html data using `beautiful_soup`.


In [ ]:
s= BeautifulSoup(html_data,'html5lib')

Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Quarterly Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.


<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


In [ ]:
l= pd.read_html(str(s))
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in s.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    #tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
    tpd = pd.DataFrame({"Date": date, "Revenue": revenue}, index=[0])
    tesla_revenue=pd.concat([tesla_revenue, tpd], ignore_index=True)
tesla_revenue.head()

Execute the following line to remove the comma and dollar sign from the `Revenue` column. 


In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

Execute the following lines to remove an null or empty strings in the Revenue column.


In [ ]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Last 5 row of the `tesla_revenue` dataframe using the `tail` function.


In [ ]:
tesla_revenue.tail(5)

## Using yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [ ]:
gm= yf.Ticker('GME')

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Setting the `period` parameter to `max` so we get information for the maximum amount of time.


In [ ]:
gme_data=gm.history(period="max")

**Resetting the index** using the `reset_index(inplace=True)` function on the gme_data DataFrame and display the first five rows of the `gme_data` dataframe using the `head` function.

In [ ]:
gme_data.reset_index(inplace=True)
gme_data.head()

## Using Webscraping to Extract GME Revenue Data


Use the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. 

In [ ]:
gd= requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html")
html_data= gd.text

Parsing the html data using `beautiful_soup`.


In [ ]:
gmesoup=BeautifulSoup(html_data, 'html5lib')

Using `BeautifulSoup` or the `read_html` function we extract the table with `GameStop Quarterly Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. The comma and dollar sign must be removed from the `Revenue` column like we did for the `tesla_revenue` dataframe.

<details><summary>Click here if you need help locating the table</summary>

```
    
Below is the code to isolate the table, you will now need to loop through the rows and columns like in the previous lab
    
soup.find_all("tbody")[1]
    
If you want to use the read_html function the table is located at index 1


```

</details>


In [ ]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in gmesoup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    gpd = pd.DataFrame({"Date": date, "Revenue": revenue}, index=[0])
    gme_revenue = pd.concat([gme_revenue, gpd], ignore_index=True)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$', "")
#gme_revenue.head()

Displaying the last five rows of the `gme_revenue` dataframe using the `tail` function.


In [ ]:
gme_revenue.tail()

## Plotting Tesla Stock Graph


We use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(tesla_data, tesla_revenue, 'Tesla')`. Note the graph will only show data upto June 2021.


In [ ]:
make_graph(tesla_data,tesla_revenue, 'TESLA') #not visible in github

## Plotting GameStop Stock Graph


We Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`. Note the graph will only show data upto June 2021.


In [ ]:
make_graph(gme_data,gme_revenue, 'GameStop')

# Conclusion:
We visualized the trend in stock prices in this project and built a plotly graph to show the trend in stock prices and revenue for Tesla and GameStop. We also used the yfinance library to get the historical stock prices for Tesla and GameStop. We used the requests library to get the revenue data for Tesla and GameStop. We used the BeautifulSoup library to parse the HTML data and extract the revenue data. We used the pandas library to create a dataframe from the revenue data. We used the plotly library to create an interactive plotly graph to visualize the trend in stock prices and revenue for Tesla and GameStop.<br>
`Ayush Priyam, Jan 2023`